In [ ]:
! git clone https://github.com/lopezjuanma96/VITS-fast-fine-tuning.git
%cd VITS-fast-fine-tuning
%pip install -r requirements.txt
# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
%cd pretrained_models
# download pretrained discriminator checkpoint
!wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth
#!wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth
!wget https://huggingface.co/lopezjm96/spanishVITS/resolve/main/G_spanish.pth -O G_trilingual.pth
%cd ..
!mkdir data

In [ ]:
import json
import os

%pip install kaggle

try:
    os.mkdir('/root/.kaggle')
except FileExistsError:
    pass

with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write(json.dumps({
        "username": "lopezjuanma96",
        "key": "b0d13c5052167d717ddb9bb06dd5f27d"
    }))

In [ ]:
!kaggle datasets download bryanpark/spanish-single-speaker-speech-dataset -f transcript.txt -p data
!unzip data/transcript.txt.zip -d data

with open('data/transcript.txt') as f:
    transcript_text = f.read()

data_file_names = [t.split('|')[0] for t in transcript_text.split('\n')]

print(data_file_names)

In [ ]:
try:
    os.mkdir('data/audio')
except FileExistsError:
    pass

use_all_files = False

if use_all_files:
    pass
else:
    data_dst = f'data/audio'
    indexes = [0] + [i for i in range(1, len(data_file_names)) if data_file_names[i-1].split('/')[0] != data_file_names[i].split('/')[0]][:-1] #-1 bc last line has empty string
    for j in indexes:
        for i in range(20): #kaggle api only accesses the first 20 files on each folder, after that it raises File Not Found
            data_src = data_file_names[j + i]

            print(data_src, data_dst)
            !kaggle datasets download bryanpark/spanish-single-speaker-speech-dataset -f {data_src} -p {data_dst}

In [ ]:
#EXTRACTED FROM preprocess.py

# load data
with open("data/transcript.txt", 'r', encoding='utf-8') as f:
    annos = f.readlines()

actual_annos_num = 0
with open("final_annotation_train.txt", 'w', encoding='utf-8') as f:
    for line in annos:
        line = line.split('|')
        line[0] = f'data/audio/{line[0].split("/")[1]}'
        if os.path.exists(line[0]):
            line = f'{line[0]}|101|[ES]{line[1]}[ES]\n'
            f.write(line)
            actual_annos_num += 1
# save annotation file for validation
with open("final_annotation_val.txt", 'w', encoding='utf-8') as f:
    for line in annos[::actual_annos_num//3]:
        line = line.split('|')
        line[0] = f'data/audio/{line[0].split("/")[1]}'
        if os.path.exists(line[0]):
            line = f'{line[0]}|101|[ES]{line[1]}[ES]\n'
            f.write(line)
            actual_annos_num += 1

In [ ]:
!python finetune_speaker.py